In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [ ]:
!pip install arabic-stopwords
!pip install qalsadi
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import torch
from torch import nn
from tqdm import tqdm
import pickle

from gru_model import ArabicDataset, Classifier, evaluate, train
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector

In [2]:
# needed functions
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Read train data

In [3]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [6]:
#remove first row that has the header
train_data['category'] = train_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,1,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,2,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,2,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,1,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,4,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",2,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,2,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,4,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,9,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,2,1


## Over Sampling

In [7]:
# !pip install imblearn

In [8]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# train_data=train_data.drop('category',axis=1)
# y=train_data['stance']
# print(Counter(train_data['stance']))
# train_data=train_data.drop('stance',axis=1)
# # define oversampling strategy
# oversample = RandomOverSampler(random_state=3)
# # fit and apply the transform
# train_data["text"], train_data['stance'] = oversample.fit_resample(train_data, y)
# print(Counter(train_data['stance']))

# Pre-Processing the tweets

In [4]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
print(train_data["text"][9])

دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية لقاح لعد ما اتابع الاخبار هم بكل مجالاتهم متفوقين وراح يطلع اللقاح قريباً؟<LF>#دعبول_دومه_مسحول
دعبول حضر من نت طلب قائد دول إسلام قاح عد تابع اخبار مجال متفوق طلع قاح قريبا دعبول دوم مسحول


## Ara2Vec Embedding

In [5]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        # preprocessed = processPost(text)
        return self.tokenizer(text)

In [7]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## create vocablary

In [8]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>')
vocab.insert(0, '<مجهول>')
word2index = {word: i for i, word in enumerate(vocab)}

In [29]:
vocab

['<مجهول>',
 'غذائ',
 'مخنوق',
 'محيط',
 'مطر',
 'خسار',
 'بريطاني',
 'ميغيل',
 'DuckDuckGo',
 'قراب',
 'جانفي',
 'يأس',
 'جزع',
 'تعرف',
 'خي',
 'وليد',
 'ضعيف',
 '٦٠',
 'متخليش',
 'حاخام',
 'جدآ',
 'تصريح',
 'حكو',
 'حديد',
 'مرتفع',
 'لبانوراما',
 '96',
 'تحف',
 'شكش',
 'مضادلمرض',
 'عجبا',
 'مامرت',
 'وبار',
 'هلأتن',
 'حدوث',
 'صنعو',
 'ختف',
 'سمعوا',
 'بقا',
 'و3',
 'وش',
 'تمام',
 'إدار',
 'اردن',
 'حبيبه',
 'تجا',
 'وظيفه',
 'معترف',
 '٪🤲🤚',
 'يفيق',
 'موروث',
 'سيطره',
 'يام',
 'امرأ',
 'لاستغلاله',
 'حث',
 'عشربن',
 '141هج',
 '75',
 'مضطر',
 'مرتش',
 'بخار',
 '٪\u061c\u200d♀️',
 'فحوص',
 'إحساس',
 '17',
 'قطع',
 '81',
 'امتحان',
 'جس',
 'عنجد',
 'إجهاد',
 'متشد',
 'مهدود',
 'قدمت',
 'الفرنسيه73',
 'يدوا',
 'ملتيميديا',
 'مركزيا',
 'تعويل',
 'غيبوب',
 'كلاسيك',
 'يستهين',
 'كي',
 'اشخاص',
 'طفئو',
 'حواش',
 'عميد',
 'الأولويةتغطية',
 'شورى',
 'وتر',
 'يحميكش',
 'وثق',
 'زرنگ',
 'إطاح',
 'يسير',
 '26يناير',
 'خاف',
 'غداد',
 'صغار',
 'تيباز',
 'إكمال',
 'شكي',
 'حم',
 'ناقش',


# Feature Extraction

## 1. TD-IDF

In [47]:
ngramdata_features, word_vectorizer = get_ngram_features(train_data)
ngramdata_features.head()

# save the model to disk
filename = 'TFIDFVectorizer.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))

LinearSVC Classifier

In [15]:
clf = LinearSVC()
pipe_tfidf = make_pipeline(word_vectorizer, clf)
pipe_tfidf.fit(train_data['text'], train_data['stance'])
print_report(pipe_tfidf, test_data['text'], test_data['stance'])

# save the model to disk
filename = 'LinearSVC_tfidf.sav'
pickle.dump(pipe_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.48      0.14      0.22        70
           0       0.45      0.28      0.34       126
           1       0.85      0.96      0.90       804

    accuracy                           0.81      1000
   macro avg       0.59      0.46      0.49      1000
weighted avg       0.78      0.81      0.78      1000

accuracy: 0.814


RandomForestClassifier

In [16]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
rf = RandomForestClassifier()
rf_tfidf = rf.fit(X_train_tfidf, train_data['stance'])
y_pred = rf_tfidf.predict(X_test_tfidf)

print_report(rf_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = 'RandomForest_tfidf.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.52      0.17      0.26        70
           0       0.44      0.23      0.30       126
           1       0.84      0.95      0.89       804

    accuracy                           0.81      1000
   macro avg       0.60      0.45      0.48      1000
weighted avg       0.77      0.81      0.77      1000

accuracy: 0.807


naive bayes

In [46]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
CNB_clr = ComplementNB()
CNB_clr_tfidf = CNB_clr.fit(X_train_tfidf, train_data['stance'])
print_report(CNB_clr_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = 'NaiveBayes_tfidf.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.30      0.44      0.36        70
           0       0.30      0.63      0.41       126
           1       0.94      0.74      0.83       804

    accuracy                           0.71      1000
   macro avg       0.52      0.61      0.53      1000
weighted avg       0.82      0.71      0.74      1000

accuracy: 0.707


## 2.CBOW

In [18]:
X_train_vect_avg, X_test_vect_avg = get_word_embedding_features(train_data, test_data)

e:\CMP\seventh term\Natural Language Processing\Arabic-Tweets-Sentiment-Classification\feature_extraction.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
e:\CMP\seventh term\Natural Language Processing\Arabic-Tweets-Sentiment-Classification\feature_extraction.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


LinearSVC Classifier

In [49]:
clf = LinearSVC(class_weight={-1:0.66, 0:0.29, 1:0.05})
clf.fit(X_train_vect_avg, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'LinearSVC_CBOW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.21      0.11      0.15        70
           0       0.27      0.17      0.21       126
           1       0.83      0.91      0.87       804

    accuracy                           0.76      1000
   macro avg       0.44      0.40      0.41      1000
weighted avg       0.72      0.76      0.74      1000

accuracy: 0.761


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier

In [20]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'RandomForest_CBOW.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.40      0.06      0.10        70
           0       0.48      0.08      0.14       126
           1       0.82      0.99      0.89       804

    accuracy                           0.81      1000
   macro avg       0.56      0.37      0.38      1000
weighted avg       0.75      0.81      0.74      1000

accuracy: 0.806


## 3.Ara2Vec

In [21]:
train_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in train_data["text"]], dtype=object)
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in test_data["text"]], dtype=object)

In [22]:
X_train_vect_avg, X_test_vect_avg = avg_word_vector(train_data_embeddings, test_data_embeddings)

LinearSVC Classifier

In [23]:
clf = LinearSVC(class_weight={-1:0.66, 0:0.29, 1:0.05})
clf.fit(X_train_vect_avg, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'LinearSVC_Ara2Vec.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.20      0.11      0.14        70
           0       0.26      0.17      0.20       126
           1       0.83      0.91      0.87       804

    accuracy                           0.76      1000
   macro avg       0.43      0.40      0.41      1000
weighted avg       0.71      0.76      0.73      1000

accuracy: 0.759


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier

In [24]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = 'RandomForest_Ara2Vec.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.40      0.06      0.10        70
           0       0.41      0.07      0.12       126
           1       0.82      0.99      0.89       804

    accuracy                           0.81      1000
   macro avg       0.54      0.37      0.37      1000
weighted avg       0.74      0.81      0.74      1000

accuracy: 0.805


## create initial embedding matrix 

In [9]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

# ArabicDataset
The class that impelements the dataset for arabic tweets

In [10]:
model = Classifier(weights_train_matrix)
print(model)

Classifier(
  (embedding): Embedding(12538, 100)
  (GRU): GRU(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=3, bias=True)
)


# Training

In [11]:
train_data_tokenized_as_num = train_data_tokenized.apply(lambda x: [word2index[word] for word in x])
# apply the same tokenization to the test set
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
train_dataset = ArabicDataset(list(train_data_tokenized_as_num), train_data['stance'] + 1, word2index['<فراغ>'])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])

In [12]:
train(model, train_dataset, epochs=10, learning_rate=0.001)

100%|██████████| 219/219 [00:37<00:00,  5.83it/s]


Epochs: 1 | Train Loss: 1.0335997639725742         | Train Accuracy: 0.756153404712677



100%|██████████| 219/219 [00:46<00:00,  4.67it/s]


Epochs: 2 | Train Loss: 1.0325850939641805         | Train Accuracy: 0.788637638092041



100%|██████████| 219/219 [00:38<00:00,  5.74it/s]


...

Epochs: 8 | Train Loss: 0.5416968296107636         | Train Accuracy: 0.7995134592056274



100%|██████████| 219/219 [00:32<00:00,  6.67it/s]


Epochs: 9 | Train Loss: 0.45136929960011346         | Train Accuracy: 0.8360045552253723



100%|██████████| 219/219 [00:28<00:00,  7.58it/s]

Epochs: 10 | Train Loss: 0.3356938823785412         | Train Accuracy: 0.8674871325492859



In [19]:
# save the model to disk
filename = './models/GRU_Ara2Vec.pth'
torch.save(model.state_dict(), filename)

# Evaluation

In [23]:
gru_model = Classifier(weights_train_matrix)
gru_model.load_state_dict(torch.load('./models/GRU_Ara2Vec.pth'))

<All keys matched successfully>

In [27]:
print(test_data_tokenized_as_num[0])
print(test_dataset[0])

[3431, 7390, 3439, 4258, 12005, 11069, 5453, 8316, 10197, 6234, 3518, 2929, 2890, 2246, 9764, 5109, 3431, 7390, 1235, 7095, 2574, 11453, 10321, 2246, 21, 4258, 12005, 3439, 5253, 5453, 8316, 11123, 7187, 3493, 2351, 5453, 2929, 3406]
(tensor([ 3431,  7390,  3439,  4258, 12005, 11069,  5453,  8316, 10197,  6234,
         3518,  2929,  2890,  2246,  9764,  5109,  3431,  7390,  1235,  7095,
         2574, 11453, 10321,  2246,    21,  4258, 12005,  3439,  5253,  5453,
         8316, 11123,  7187,  3493,  2351,  5453,  2929,  3406, 12537, 12537,
        12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537,
        12537, 12537, 12537, 12537], dtype=torch.int32), tensor(2))


In [24]:
evaluate(gru_model, test_dataset)

100%|██████████| 32/32 [00:01<00:00, 30.68it/s]


accuracy: 0.730
              precision    recall  f1-score   support

           0       0.32      0.27      0.29        70
           1       0.30      0.60      0.40       126
           2       0.92      0.79      0.85       804

    accuracy                           0.73      1000
   macro avg       0.51      0.55      0.51      1000
weighted avg       0.80      0.73      0.76      1000


Test Accuracy: 0.7300000190734863


array([[-0.25917843,  1.371456  , -1.413901  ],
       [-3.7941768 ,  0.1537084 ,  4.186266  ],
       [-4.1633673 ,  0.1650067 ,  4.571656  ],
       ...,
       [-3.1534357 ,  0.28026307,  3.231864  ],
       [-0.9125517 ,  0.8395538 , -0.2619845 ],
       [-1.263289  ,  0.06857997,  1.1880075 ]], dtype=float32)